In [ ]:
import torch
import lightning
from torch import nn
from torch.nn import functional as F


class AutoEncoder(lightning.LightningModule):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 3)
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 128),
            nn.ReLU(),
            nn.Linear(128, 28 * 28)
        )

    def forward(self, x):
        embedding = self.encoder(x)
        return embedding

    def training_step(self, batch):
        x, _ = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [ ]:
import torchvision
from torch.utils.data import random_split


dataset = torchvision.datasets.MNIST(
    "../datasets", download=True, transform=torchvision.transforms.ToTensor()
)
train, valid = random_split(dataset, [55000, 5000])
print(dataset)

In [ ]:
from torch.utils.data import DataLoader


autoencoder = AutoEncoder()
trainer = lightning.Trainer(accelerator="gpu", devices=1, max_epochs=1)
trainer.fit(autoencoder, DataLoader(train), DataLoader(valid))